In [153]:
# Modules
import subprocess
from IPython.display import clear_output
import pandas as pd
import re
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import ast
from scipy.stats import norm
from scipy.optimize import minimize


In [154]:
# compile zdplaskin
def run_prep(inp_path):
    try:
        process = subprocess.Popen(
            'preprocessor.exe',
            stdout = subprocess.DEVNULL,        # ignore outputs
            stderr = subprocess.DEVNULL,        # ignore errors
            stdin = subprocess.PIPE,            # recognize input
            universal_newlines=True
        )
        
        process.stdin.write(inp_path)
        process.stdin.flush()                   # send a data

        while process.poll() is None:           # check the program state, if None, program is still in the run
            process.stdin.write('.\n')
            process.stdin.flush()
    except:
        pass
    print('check the run of preprocessor')
    return process

In [155]:
# Compile exe
def compile_zdp(name):
    compile_command = [
        'gfortran', '-o', name, 'dvode_f90_m.F90', 'zdplaskin_m.F90',
        'run_plasRxn_v2.F90', 'bolsig_x86_64_g.dll'
    ]
    
    try:
        subprocess.run(compile_command)
    except:
        pass
    print('check the compiler')

In [156]:
# Run exe
def run_exe(exe_path):
    try:
        process = subprocess.Popen(
            exe_path,
            stdout = subprocess.PIPE, # Read standard outputs
            stderr = subprocess.PIPE, # Read standard errors
            universal_newlines=True,  # outputs to str variables
            bufsize = 1               # control the size of buffer
        )

        log_flag = False             # The flag for starting log after "Caculation Start!!"
        while True:
            output = process.stdout.readline()
            if not output:
                break
            if "Calculation Start" in output:
                log_flag = True

            if log_flag:
                print(f'\r{output.strip()}           ',end='',flush=True)

            if "PRESS ENTER TO EXIT" in output:
                process.kill()        # forced shutdown
                break
            if "WARNING: BOLSIG+ convergence failed" in output:
                process.kill()        # forced shutdown
                break
    except:
        pass
    return process

In [157]:
# Error calculation
def cal_error(exp_result):
    # Read a result
    conditions = []
    with open('qt_conditions_list.txt','r') as file:
        for line in file:
            line = line.strip()
            line = line[2:]
            conditions.append(line)
        file.close()

    species = []
    with open('qt_species_list.txt','r') as file:
        for line in file:
            line = line.rstrip()
            line = line[3:]
            species.append(line)
        file.close()

    reactions = []
    reaction_list = pd.read_csv('parameter_set.csv')
    reactions = reaction_list['Reaction'].to_list()
    df_cd = pd.read_csv('qt_conditions.txt', sep=r'\s+', header=0, names=['Time [s]']+conditions)
    df_sp = pd.read_csv('qt_densities.txt', sep=r'\s+', header=0, names=['Time [s]']+species)
    df_rx = pd.read_csv('qt_rates.txt', sep=r'\s+', header=0, names=['Time [s]']+reactions)
    top_rate = df_rx.iloc[:,1:].sum().sort_values(ascending=False)

    CH4 = (df_sp['CH4'] + df_sp['CH4(V13)'] + df_sp['CH4(V24)'])
    C2H2 = (df_sp['C2H2'] + df_sp['C2H2(V13)']+ df_sp['C2H2(V2)']+ df_sp['C2H2(V5)'])
    C2H4 = (df_sp['C2H4'] + df_sp['C2H4(V1)']+ df_sp['C2H4(V2)'])
    C2H6 = (df_sp['C2H6'] + df_sp['C2H6(V13)']+ df_sp['C2H6(V24)'])
    C3H6 = (df_sp['C3H6'] + df_sp['C3H6(V)'])
    C3H8 = (df_sp['C3H8'] + df_sp['C3H8(V1)'] + df_sp['C3H8(V2)'])
    C4H10 = (df_sp['C4H9H'])
    C5H12 = (df_sp['C5H12'])
    H2 = df_sp['H2']
    C = df_sp['C']
    H = df_sp['H']
    CH = df_sp['CH']
    CH2 = df_sp['CH2']
    CH3 = df_sp['CH3']
    C2H3 = df_sp['C2H3']
    C2H5 = df_sp['C2H5']
    C3H5 = df_sp['C3H5']
    C3H7 = df_sp['C3H7']
    C4H9 = df_sp['C4H9']

    exp = exp_result
    
    sim_XCH4 = (CH4.iloc[0] - CH4.iloc[-1])/CH4.iloc[0]*100
    sim_SH2 = 0.5*H2.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H6 = 2*C2H6.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H4 = 2*C2H4.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H2 = 2*C2H2.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H8 = 3*C3H8.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H6 = 3*C3H6.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC4H10 = 4*C4H10.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC5H12 = 5*C5H12.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SH = 0.25 * H.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SCH3 = CH3.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SCH2 = CH2.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H3 = 2*C2H3.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H5 = 2*C2H5.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H5 = 3*C3H5.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H7 = 3*C3H7.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC4H9 = 4*C4H9.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    # sim_SH2 = H2.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H6 = C2H6.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H4 = C2H4.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H2 = C2H2.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC3H8 = C3H8.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC3H6 = C3H6.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC4H10 = C4H10.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC5H12 = C5H12.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SH = H.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SCH = CH.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SCH2 = CH2.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SCH3 = CH3.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H3 = C2H3.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H5 = C2H5.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC3H5 = C3H5.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC3H7 = C3H7.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC4H9 = C4H9.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100


    sim = []
    sim.append(sim_XCH4)
    sim.append(sim_SH2)
    sim.append(sim_SC2H6)
    sim.append(sim_SC2H4)
    sim.append(sim_SC2H2)
    sim.append(sim_SC3H8)
    sim.append(sim_SC3H6)
    sim.append(sim_SC4H10)
    sim.append(sim_SC5H12)
    sim.append(sim_SH)
    sim.append(sim_SCH3)
    sim.append(sim_SCH2)
    sim.append(sim_SC2H3)
    sim.append(sim_SC2H5)
    sim.append(sim_SC3H5)
    sim.append(sim_SC3H7)
    sim.append(sim_SC4H9)

    sim2 = []
    sim2.append(sim_XCH4)
    sim2.append(sim_SH2)
    sim2.append(sim_SC2H6)
    sim2.append(sim_SC2H4)
    sim2.append(sim_SC2H2)
    sim2.append(sim_SC3H8)
    sim2.append(sim_SC3H6)
    sim2.append(sim_SC4H10)
    sim2.append(sim_SH)
    sim2.append(sim_SCH3)
    sim2.append(sim_SCH2)
    sim2.append(sim_SC2H3)
    sim2.append(sim_SC2H5)
    sim2.append(sim_SC3H5)
    sim2.append(sim_SC3H7)
    sim2.append(sim_SC4H9)

    err =  float(np.sum((np.asarray(exp) - np.asarray(sim2))**2))
    
    return err, top_rate, sim, df_cd['Time [s]'].iloc[-1],df_sp.iloc[-1].sort_values(ascending=False)

    

In [158]:
result_list = ['XCH4','SH2','SC2H6','SC2H4','SC2H2','SC3H8','SC3H6','SC4','SC5+','SH','SCH3','SCH2','SC2H3','SC2H5','SC3H5','SC3H7','SC4H9']
exp_result = [16.1,77.031,12.999,1.535,1.302,3.499,0.467,0.422,0,0,0,0,0,0,0,0]
exp_result_expr = [16.1,77.031,12.999,1.535,1.302,3.499,0.467,0.422,2.741,0,0,0,0,0,0,0,0]

In [159]:
len(exp_result_expr)

17

In [160]:
inp_path = f'kinet12`.inp\n'
exe_path = 'run_plasRxn_v2.exe'
prep_process = run_prep(inp_path)
prep_process.wait()
compile_zdp(exe_path)
exe_process = run_exe(exe_path)
exe_process.wait()
error, top_rate, sim_result, total_time, total_species = cal_error(exp_result)

check the run of preprocessor
check the compiler
PRESS ENTER TO EXIT ...           (s)           gured for BOLSIG+ solver exceeds 1.78D-01           

In [161]:
sim_result2 = [round(float(i),3) for i in sim_result]

In [162]:
df_res = pd.DataFrame({
    'res': result_list,
    'exp': exp_result_expr,
    'sim': sim_result2
    }
)
display(df_res)
print(total_time)
print(error)

,res,exp,sim
0,XCH4,16.100,15.968
1,SH2,77.031,77.333
2,SC2H6,12.999,12.848
3,SC2H4,1.535,1.557
4,SC2H2,1.302,1.249
5,SC3H8,3.499,3.624
6,SC3H6,0.467,0.407
7,SC4,0.422,0.791
8,SC5+,2.741,14.560
9,SH,0.000,0.016


2.1186
0.2968697855390603


In [163]:
print(top_rate[:50])

H + H => H2                          3.402355e+21
e + CH => e + C + H                  2.828961e+21
e + CH4 => e + CH + H2 + H           2.819263e+21
e + CH4 => e + e + H + CH3^+         6.717197e+20
e + CH4 => e + CH3 + H               6.212439e+20
e + e + CH3^+ => e + CH3             5.698193e+20
CH3 + CH3 => C2H6                    5.547971e+20
e + C2H6 => e + C2H4 + H2            1.765030e+20
e + CH4 => e + CH4(v24)              1.718997e+20
C2H4 + H => C2H5                     1.340081e+20
e + CH4 => e + CH2 + H2              1.262219e+20
e + CH4 => e + CH4(v13)              1.048928e+20
CH4 + CH3^+ => H2 + C2H5^+           1.038688e+20
e + e + C2H5^+ => e + C2H5           9.846915e+19
C2H5 + C2H5 => C4H9H                 9.456104e+19
C4H9H + CH2 => C5H12                 8.502905e+19
CH3 + C2H5 => C3H8                   4.192493e+19
e + C2H4 => e + C2H2 + H2            1.628222e+19
e + CH4(v24) => e + CH + H2 + H      5.999556e+18
e + C2H5^+ => C2H3 + H + H           5.763216e+18


In [164]:
print(total_species[:50])

CH4          2.047300e+19
H2           6.085000e+18
C            2.551900e+18
C2H6         2.511600e+17
CH4(V24)     1.436900e+17
C5H12        1.145700e+17
CH4(V13)     8.800400e+16
C3H8         4.689800e+16
C2H4         3.061800e+16
C2H2         2.451600e+16
C4H9H        7.783300e+15
C3H6         5.304200e+15
CH3          3.073100e+15
H            2.507500e+15
C2H6(V24)    8.559300e+14
C2H6(V13)    7.235700e+14
C3H8(V1)     3.434700e+14
C2H5         2.927700e+14
C3H8(V2)     2.894100e+14
C3H6(V)      3.406800e+13
C2H2(V5)     3.076500e+13
C2H3         1.948400e+13
C2H4(V1)     1.595400e+13
CH2          1.221600e+13
C2H2(V2)     8.825800e+12
C2H2(V13)    6.188900e+12
C2H4(V2)     4.427500e+12
CH           6.493200e+09
E            6.221900e+05
CH3^+        5.278600e+05
C2H5^+       9.432700e+04
Time [s]     2.118600e+00
C3H7         2.500000e-02
C3H5         3.438000e-09
H^+          2.235400e-10
C4H9         2.613100e-13
C2H          3.756100e-15
CH2^+        1.309800e-15
CH4^+       